<a href="https://colab.research.google.com/github/sid-0837/siddharth/blob/main/Football_Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup

In [2]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020','2021','2022']

In [3]:
# Starting with latest data for Spanish league
url = base_url+'/'+leagues[0]+'/'+seasons[4]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

# Based on the structure of the webpage, I found that data is in the JSON variable
scripts = soup.find_all('script')



In [4]:
import json

string_with_json_obj = ''

# Find data for teams
for el in scripts:
    if 'teamsData' in el.text:
      string_with_json_obj = el.text.strip()

# print(string_with_json_obj)

# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

In [5]:
# convert JSON data into Python dictionary
data = json.loads(json_data)
print(data.keys())
print('='*50)
print(data['138'].keys())
print('='*50)
print(data['138']['id'])
print('='*50)
print(data['138']['title'])
print('='*50)
print(data['138']['history'][0])

# Print pretty JSON data to check out what we have there
# s = json.dumps(data, indent=4, sort_keys=True)
# print(s)

dict_keys(['138', '140', '141', '142', '143', '145', '146', '147', '148', '150', '151', '152', '153', '154', '156', '158', '159', '223', '231', '232'])
dict_keys(['id', 'title', 'history'])
138
Sevilla
{'h_a': 'a', 'xG': 3.34997, 'xGA': 2.39239, 'npxG': 3.34997, 'npxGA': 1.64976, 'ppda': {'att': 252, 'def': 19}, 'ppda_allowed': {'att': 263, 'def': 20}, 'deep': 12, 'deep_allowed': 5, 'scored': 4, 'missed': 1, 'xpts': 1.9829999999999999, 'result': 'w', 'date': '2018-08-19 21:15:00', 'wins': 1, 'draws': 0, 'loses': 0, 'pts': 3, 'npxGD': 1.70021}


In [6]:
# Get teams and their relevant ids and put them into separate dictionary
teams = {}
for id in data.keys():
  teams[id] = data[id]['title']

In [7]:
# EDA to get a feeling of how the JSON is structured
# Column names are all the same, so we just use first element
columns = []
# Check the sample of values per each column
values = []
for id in data.keys():
  columns = list(data[id]['history'][0].keys())
  values = list(data[id]['history'][0].values())
  break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['a', 3.34997, 2.39239, 3.34997, 1.64976, {'att': 252, 'def': 19}, {'att': 263, 'def': 20}, 12, 5, 4, 1, 1.9829999999999999, 'w', '2018-08-19 21:15:00', 1, 0, 0, 3, 1.70021]


In [8]:
sevilla_data = []
for row in data['138']['history']:
  sevilla_data.append(list(row.values()))

df = pd.DataFrame(sevilla_data, columns=columns)
df.head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,1,1.9830,w,2018-08-19 21:15:00,1,0,0,3,1.700210
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,0,2.3331,d,2018-08-26 21:15:00,0,1,0,1,1.300181


In [9]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
  teams_data = []
  for row in data[id]['history']:
    teams_data.append(list(row.values()))

  df = pd.DataFrame(teams_data, columns=columns)
  dataframes[team] = df
  print('Added data for {}.'.format(team))

Added data for Sevilla.
Added data for Real Sociedad.
Added data for Espanyol.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Real Madrid.
Added data for Levante.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Eibar.
Added data for Alaves.
Added data for Leganes.
Added data for Girona.
Added data for Real Valladolid.
Added data for SD Huesca.


In [10]:
# Sample check of our newly created DataFrame
dataframes['Barcelona'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,h,3.26753,0.248353,3.26753,0.248353,"{'att': 118, 'def': 17}","{'att': 407, 'def': 13}",20,0,3,0,2.9009,w,2018-08-18 23:15:00,1,0,0,3,3.019177
1,a,1.20392,0.510742,1.20392,0.510742,"{'att': 163, 'def': 16}","{'att': 316, 'def': 15}",15,4,1,0,1.9865,w,2018-08-25 23:15:00,1,0,0,3,0.693178


In [11]:
for team, df in dataframes.items(): #Manipulations to make data as in the original source
    dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['ppda_att'] = dataframes[team]['ppda'].apply(lambda x: x['att'])
    dataframes[team]['ppda_def'] = dataframes[team]['ppda'].apply(lambda x: x['def'])
    dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['oppda_att'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att'])
    dataframes[team]['oppda_def'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['def'])

# And check how our new dataframes look based on Sevilla dataframe
dataframes['Sevilla'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,draws,loses,pts,npxGD,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def
0,a,3.34997,2.392390,3.34997,1.649760,"{'att': 252, 'def': 19}","{'att': 263, 'def': 20}",12,5,4,...,0,0,3,1.700210,13.263158,252,19,13.150000,263,20
1,h,1.97161,0.671429,1.97161,0.671429,"{'att': 262, 'def': 16}","{'att': 237, 'def': 26}",11,3,0,...,1,0,1,1.300181,16.375000,262,16,9.115385,237,26


In [12]:
frames = []
for team, df in dataframes.items():
    df['team'] = team
    frames.append(df)

full_stat = pd.concat(frames)
full_stat = full_stat.drop(['ppda', 'ppda_allowed'], axis=1)

In [13]:
full_stat.head(10)

,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,...,loses,pts,npxGD,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team
0,a,3.349970,2.392390,3.349970,1.649760,12,5,4,1,1.9830,...,0,3,1.700210,13.263158,252,19,13.150000,263,20,Sevilla
1,h,1.971610,0.671429,1.971610,0.671429,11,3,0,0,2.3331,...,0,1,1.300181,16.375000,262,16,9.115385,237,26,Sevilla
2,a,0.784969,1.182950,0.784969,1.182950,2,5,0,1,0.9556,...,1,0,-0.397981,12.120000,303,25,7.095238,149,21,Sevilla
3,h,1.082550,2.127170,1.082550,2.127170,20,4,0,2,0.6338,...,1,0,-1.044620,4.115385,107,26,13.409091,295,22,Sevilla
4,a,3.401280,2.248470,3.401280,1.505320,13,9,6,2,2.1285,...,0,3,1.895960,10.217391,235,23,17.800000,267,15,Sevilla
5,h,2.292620,1.553510,2.292620,1.553510,7,10,3,0,1.9024,...,0,3,0.739110,17.647059,300,17,12.294118,209,17,Sevilla
6,a,1.985320,0.563865,1.242040,0.563865,2,2,3,1,2.5246,...,0,3,0.678175,23.909091,263,11,4.676471,159,34,Sevilla
7,h,2.237000,0.895831,2.237000,0.895831,8,9,2,1,2.3711,...,0,3,1.341169,11.000000,242,22,12.238095,257,21,Sevilla
8,a,1.898010,3.219480,1.898010,2.476200,8,8,2,4,0.6312,...,1,0,-0.578190,10.321429,289,28,10.714286,225,21,Sevilla
9,h,1.884310,1.013060,1.884310,1.013060,9,5,2,1,2.0721,...,0,3,0.871250,11.937500,191,16,18.166667,327,18,Sevilla


In [14]:
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

In [15]:
full_stat.head()

,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,...,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team,xG_diff,xGA_diff,xpts_diff
0,a,3.349970,2.392390,3.349970,1.649760,12,5,4,1,1.9830,...,13.263158,252,19,13.150000,263,20,Sevilla,-0.650030,1.392390,-1.0170
1,h,1.971610,0.671429,1.971610,0.671429,11,3,0,0,2.3331,...,16.375000,262,16,9.115385,237,26,Sevilla,1.971610,0.671429,1.3331
2,a,0.784969,1.182950,0.784969,1.182950,2,5,0,1,0.9556,...,12.120000,303,25,7.095238,149,21,Sevilla,0.784969,0.182950,0.9556
3,h,1.082550,2.127170,1.082550,2.127170,20,4,0,2,0.6338,...,4.115385,107,26,13.409091,295,22,Sevilla,1.082550,0.127170,0.6338
4,a,3.401280,2.248470,3.401280,1.505320,13,9,6,2,2.1285,...,10.217391,235,23,17.800000,267,15,Sevilla,-2.598720,0.248470,-0.8715


In [16]:
season_data = dict() #Scraping data for all teams of all leagues of all seasons
season_data[seasons[4]] = full_stat
print(season_data)
full_data = dict()
full_data[leagues[0]] = season_data
print(full_data)

{'2018':    h_a        xG       xGA      npxG     npxGA  deep  deep_allowed  scored  \
0    a  3.349970  2.392390  3.349970  1.649760    12             5       4   
1    h  1.971610  0.671429  1.971610  0.671429    11             3       0   
2    a  0.784969  1.182950  0.784969  1.182950     2             5       0   
3    h  1.082550  2.127170  1.082550  2.127170    20             4       0   
4    a  3.401280  2.248470  3.401280  1.505320    13             9       6   
..  ..       ...       ...       ...       ...   ...           ...     ...   
33   h  1.193850  0.651098  1.193850  0.651098     4             4       2   
34   a  1.330980  1.158080  1.330980  1.158080     1             8       1   
35   h  2.417820  2.608680  1.674550  2.608680     5             8       2   
36   a  1.517810  2.009140  0.774535  2.009140     1            10       1   
37   h  1.304680  0.916693  1.304680  0.916693     3             3       2   

    missed    xpts  ...  ppda_coef ppda_att  ppda_def 

In [17]:
full_data = dict()
for league in leagues:

  season_data = dict()
  for season in seasons:
    url = base_url+'/'+league+'/'+season
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "lxml")

    # Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
    scripts = soup.find_all('script')

    string_with_json_obj = ''

    # Find data for teams
    for el in scripts:
        if 'teamsData' in el.text:
          string_with_json_obj = el.text.strip()

    # print(string_with_json_obj)

    # strip unnecessary symbols and get only JSON data
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')


    # convert JSON data into Python dictionary
    data = json.loads(json_data)

    # Get teams and their relevant ids and put them into separate dictionary
    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']

    # EDA to get a feeling of how the JSON is structured
    # Column names are all the same, so we just use first element
    columns = []
    # Check the sample of values per each column
    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break

    # Getting data for all teams
    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df
      # print('Added data for {}.'.format(team))


    for team, df in dataframes.items():
        dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
        dataframes[team]['ppda_att'] = dataframes[team]['ppda'].apply(lambda x: x['att'])
        dataframes[team]['ppda_def'] = dataframes[team]['ppda'].apply(lambda x: x['def'])
        dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
        dataframes[team]['oppda_att'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att'])
        dataframes[team]['oppda_def'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['def'])

    frames = []
    for team, df in dataframes.items():
        df['team'] = team
        frames.append(df)

    full_stat = pd.concat(frames)
    full_stat = full_stat.drop(['ppda', 'ppda_allowed'], axis=1)

    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']

    full_stat.reset_index(inplace=True, drop=True)
    season_data[season] = full_stat

  df_season = pd.concat(season_data)
  full_data[league] = df_season

data = pd.concat(full_data)
data.head()



h_a        xG       xGA      npxG     npxGA  deep  \
La_liga 2014 0   h  1.321070  1.141510  0.438073  1.141510     4   
             1   a  1.131960  1.968630  1.131960  1.069640     4   
             2   h  0.819725  0.358265  0.819725  0.358265     7   
             3   a  0.446347  2.761880  0.446347  2.761880     6   
             4   h  0.278657  0.646364  0.278657  0.646364     4   

                deep_allowed  scored  missed    xpts  ...  ppda_coef ppda_att  \
La_liga 2014 0             5       1       0  1.5303  ...  12.071429      338   
             1             5       0       3  0.7086  ...  15.368421      292   
             2             1       0       0  1.7526  ...   9.846154      128   
             3             2       2       2  0.0628  ...   5.500000       99   
             4             7       0       0  0.8721  ...   9.451613      293   

                ppda_def  oppda_coef  oppda_att  oppda_def    team   xG_diff  \
La_liga 2014 0        28    6.300000        189         30  Malaga  0.321070   
             1        19    6.066667        182         30  Malaga  1.131960   
             2        13    8.925926        241         27  Malaga  0.819725   
             3        18    8.000000        264         33  Malaga -1.553653   
             4        31    4.571429         96         21  Malaga  0.278657   

                xGA_diff  xpts_diff  
La_liga 2014 0  1.141510    -1.4697  
             1 -1.031370     0.7086  
             2  0.358265     0.7526  
             3  0.761880    -0.9372  
             4  0.646364    -0.1279  

[5 rows x 27 columns]

In [18]:
data.index = data.index.droplevel(2)
data.index = data.index.rename(names=['league','year'], level=[0,1])
data.head()

h_a        xG       xGA      npxG     npxGA  deep  deep_allowed  \
league  year                                                                   
La_liga 2014   h  1.321070  1.141510  0.438073  1.141510     4             5   
        2014   a  1.131960  1.968630  1.131960  1.069640     4             5   
        2014   h  0.819725  0.358265  0.819725  0.358265     7             1   
        2014   a  0.446347  2.761880  0.446347  2.761880     6             2   
        2014   h  0.278657  0.646364  0.278657  0.646364     4             7   

              scored  missed    xpts  ...  ppda_coef ppda_att  ppda_def  \
league  year                          ...                                 
La_liga 2014       1       0  1.5303  ...  12.071429      338        28   
        2014       0       3  0.7086  ...  15.368421      292        19   
        2014       0       0  1.7526  ...   9.846154      128        13   
        2014       2       2  0.0628  ...   5.500000       99        18   
        2014       0       0  0.8721  ...   9.451613      293        31   

              oppda_coef  oppda_att  oppda_def    team   xG_diff  xGA_diff  \
league  year                                                                 
La_liga 2014    6.300000        189         30  Malaga  0.321070  1.141510   
        2014    6.066667        182         30  Malaga  1.131960 -1.031370   
        2014    8.925926        241         27  Malaga  0.819725  0.358265   
        2014    8.000000        264         33  Malaga -1.553653  0.761880   
        2014    4.571429         96         21  Malaga  0.278657  0.646364   

              xpts_diff  
league  year             
La_liga 2014    -1.4697  
        2014     0.7086  
        2014     0.7526  
        2014    -0.9372  
        2014    -0.1279  

[5 rows x 27 columns]

In [19]:
data.tail()

h_a        xG       xGA      npxG     npxGA  deep  deep_allowed  \
league year                                                                   
RFPL   2022   a  0.474894  0.489886  0.474894  0.489886     4            12   
       2022   h  1.576520  0.264197  0.822241  0.264197     3             6   
       2022   a  1.243320  0.771737  1.243320  0.771737     4             6   
       2022   h  1.404960  0.550495  1.404960  0.550495     5             4   
       2022   a  0.229928  1.125040  0.229928  1.125040     4             6   

             scored  missed    xpts  ...  ppda_coef ppda_att  ppda_def  \
league year                          ...                                 
RFPL   2022       2       0  1.2515  ...  14.866667      223        15   
       2022       1       0  2.6044  ...   5.964286      167        28   
       2022       0       1  1.7720  ...  16.941176      288        17   
       2022       2       1  2.1503  ...   7.909091      174        22   
       2022       0       1  0.4989  ...  10.684211      406        38   

             oppda_coef  oppda_att  oppda_def   team   xG_diff  xGA_diff  \
league year                                                                
RFPL   2022    4.571429         96         21  Fakel -1.525106  0.489886   
       2022   11.600000        116         10  Fakel  0.576520  0.264197   
       2022    4.812500         77         16  Fakel  1.243320 -0.228263   
       2022    4.800000         72         15  Fakel -0.595040 -0.449505   
       2022    4.352941         74         17  Fakel  0.229928  0.125040   

             xpts_diff  
league year             
RFPL   2022    -1.7485  
       2022    -0.3956  
       2022     1.7720  
       2022    -0.8497  
       2022     0.4989  

[5 rows x 27 columns]

In [20]:
data.to_csv('Web_Scraping_Football_Statistics_understat_per_game.csv')